In [18]:
import cv2
import mediapipe as mp
import numpy as np
import time
import pyttsx3
mp_drawing = mp.solutions.drawing_utils #my notes=  to visulize the lines on the body 
mp_pose = mp.solutions.pose # from solutions we want the pose estemation model

In [29]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0]) #y2-y1 , x2-x1 , here we calculated the dgree in radians
    angle = np.abs(radians*180.0/np.pi) # we converted radiance to dgree
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [50]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate angle (not used here, but you can keep it for reference)
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arccos(
        np.clip(np.dot(a - b, c - b) / (np.linalg.norm(a - b) * np.linalg.norm(c - b)), -1.0, 1.0)
    )
    return np.degrees(radians)

cap = cv2.VideoCapture(0)

counter = 0
stage = "center"

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Flip the image to mirror the webcam feed

        # Convert the image to RGB (required by MediaPipe)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of relevant landmarks
            NOSE = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                    landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            # Midpoint of the shoulders (used for head direction)
            shoulder_mid_x = (LEFT_SHOULDER[0] + RIGHT_SHOULDER[0]) / 2
            threshold = 0.15  # Adjust this value for sensitivity

            # Detect direction based on nose's x-position
            if stage == "center":
                if NOSE[0] < (shoulder_mid_x - threshold):
                    stage = "left"  # Head turned left
                elif NOSE[0] > (shoulder_mid_x + threshold):
                    stage = "right"  # Head turned right

            elif stage in ["left", "right"]:
                if (shoulder_mid_x - threshold) <= NOSE[0] <= (shoulder_mid_x + threshold):
                    stage = "center"  # Head back to center
                    counter += 1  # Increment rep count
                    print("Reps:", counter)

            # Debug: Draw the midpoint line and nose
            mid_x_pixel = int(shoulder_mid_x * 640)  # Convert to pixel coordinates
            #cv2.line(image, (mid_x_pixel, 0), (mid_x_pixel, 480), (0, 255, 0), 2)  # Midpoint line

            nose_pixel = tuple(np.multiply(NOSE, [640, 480]).astype(int))  # Nose pixel
            cv2.circle(image, nose_pixel, 5, (255, 0, 0), -1)  # Draw nose position

        except:
            pass

        # Counter box for displaying rep count
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)

        # Display rep count
        cv2.putText(image, 'REPS', (15, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, str(counter), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        # Display stage (center, left, right)
        cv2.putText(image, 'STAGE', (65, 12),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        cv2.putText(image, stage, (60, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2)

        # Draw pose landmarks for visual feedback
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Show the webcam feed with the annotations
        cv2.imshow('Mediapipe Feed', image)

        # Break the loop when 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


Reps: 1
Reps: 2
Reps: 3
Reps: 4


i added a full reb cycle " moving from left/right to right/left " 

In [56]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate angle (not used here, but you can keep it for reference)
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arccos(
        np.clip(np.dot(a - b, c - b) / (np.linalg.norm(a - b) * np.linalg.norm(c - b)), -1.0, 1.0)
    )
    return np.degrees(radians)

cap = cv2.VideoCapture(0)

left_count = 0
center_count = 0
right_count = 0
cycle_count = 0  # Track number of complete cycles
stage = "center"

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Flip the image to mirror the webcam feed

        # Convert the image to RGB (required by MediaPipe)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of relevant landmarks
            NOSE = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                    landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            # Midpoint of the shoulders (used for head direction)
            shoulder_mid_x = (LEFT_SHOULDER[0] + RIGHT_SHOULDER[0]) / 2
            threshold = 0.15  # Adjust this value for sensitivity

            # Detect direction based on nose's x-position
            if stage == "center":
                if NOSE[0] < (shoulder_mid_x - threshold):
                    stage = "left"  # Head turned left
                    left_count += 1  # Increment left count
                elif NOSE[0] > (shoulder_mid_x + threshold):
                    stage = "right"  # Head turned right
                    right_count += 1  # Increment right count

            elif stage == "left":
                if (shoulder_mid_x - threshold) <= NOSE[0] <= (shoulder_mid_x + threshold):
                    stage = "center"  # Head back to center
                    center_count += 1  # Increment center count

            elif stage == "right":
                if (shoulder_mid_x - threshold) <= NOSE[0] <= (shoulder_mid_x + threshold):
                    stage = "center"  # Head back to center
                    center_count += 1  # Increment center count

            # If total counts for left, center, and right reach 3, we reset the counts
            total_count = left_count + center_count + right_count
            if total_count == 4:
                cycle_count += 1  # Increment cycle count
                left_count = 0  # Reset left count
                center_count = 0  # Reset center count
                right_count = 0  # Reset right count

        except:
            pass

        # Display the movement counts and cycle count
        cv2.rectangle(image, (0, 0), (225, 200), (245, 117, 16), -1)

        # Display left, center, right counts and cycle count
        cv2.putText(image, 'Left: ' + str(left_count), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, 'Center: ' + str(center_count), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, 'Right: ' + str(right_count), (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, 'Cycles: ' + str(cycle_count), (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        # Draw pose landmarks for visual feedback
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Show the webcam feed with the annotations
        cv2.imshow('Mediapipe Feed', image)

        # Break the loop when 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


i added timer

In [6]:
import cv2
import mediapipe as mp
import numpy as np
import time  # Import the time module

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

# Function to calculate angle (not used here, but you can keep it for reference)
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arccos(
        np.clip(np.dot(a - b, c - b) / (np.linalg.norm(a - b) * np.linalg.norm(c - b)), -1.0, 1.0)
    )
    return np.degrees(radians)

cap = cv2.VideoCapture(0)

left_count = 0
center_count = 0
right_count = 0
cycle_count = 0  # Track number of complete cycles
stage = "center"

# Variables for the countdown timer
countdown_time = 5  
countdown_started = False
countdown_end_time = 0

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        frame = cv2.flip(frame, 1)  # Flip the image to mirror the webcam feed

        # Convert the image to RGB (required by MediaPipe)
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        results = pose.process(image)

        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        try:
            landmarks = results.pose_landmarks.landmark

            # Get coordinates of relevant landmarks
            NOSE = [landmarks[mp_pose.PoseLandmark.NOSE.value].x,
                    landmarks[mp_pose.PoseLandmark.NOSE.value].y]
            LEFT_SHOULDER = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            RIGHT_SHOULDER = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]

            # Midpoint of the shoulders (used for head direction)
            shoulder_mid_x = (LEFT_SHOULDER[0] + RIGHT_SHOULDER[0]) / 2
            threshold = 0.15  # Adjust this value for sensitivity

            # Detect direction based on nose's x-position
            if stage == "center":
                if NOSE[0] < (shoulder_mid_x - threshold):
                    stage = "left"  # Head turned left
                    left_count += 1  # Increment left count
                    countdown_started = True
                    countdown_end_time = time.time() + countdown_time  # Start countdown

                elif NOSE[0] > (shoulder_mid_x + threshold):
                    stage = "right"  # Head turned right
                    right_count += 1  # Increment right count
                    countdown_started = True
                    countdown_end_time = time.time() + countdown_time  # Start countdown

            elif stage == "left":
                if (shoulder_mid_x - threshold) <= NOSE[0] <= (shoulder_mid_x + threshold):
                    stage = "center"  # Head back to center
                    center_count += 1  # Increment center count
                    countdown_started = False  # Stop countdown when moving back to center

            elif stage == "right":
                if (shoulder_mid_x - threshold) <= NOSE[0] <= (shoulder_mid_x + threshold):
                    stage = "center"  # Head back to center
                    center_count += 1  # Increment center count
                    countdown_started = False  # Stop countdown when moving back to center

            # If total counts for left, center, and right reach 3, we reset the counts
            total_count = left_count + center_count + right_count
            if total_count == 4:
                cycle_count += 1  # Increment cycle count
                left_count = 0  # Reset left count
                center_count = 0  # Reset center count
                right_count = 0  # Reset right count

        except:
            pass

        # Display the movement counts and cycle count
        cv2.rectangle(image, (0, 0), (225, 200), (245, 117, 16), -1)

        # Display left, center, right counts and cycle count
        cv2.putText(image, 'Left: ' + str(left_count), (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, 'Center: ' + str(center_count), (10, 60),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, 'Right: ' + str(right_count), (10, 90),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
        cv2.putText(image, 'Cycles: ' + str(cycle_count), (10, 120),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        # Show countdown timer if moving left or right
        if countdown_started:
            remaining_time = int(countdown_end_time - time.time())
            if remaining_time > 0:
                minutes = remaining_time // 60
                seconds = remaining_time % 60
                countdown_text = f"{minutes:02}:{seconds:02}"
            else:
                countdown_text = "00:00"  # Countdown finished
            cv2.putText(image, f"Countdown: {countdown_text}", (10, 150),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        # Draw pose landmarks for visual feedback
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

        # Show the webcam feed with the annotations
        cv2.imshow('Mediapipe Feed', image)

        # Break the loop when 'q' key is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()


what to do next : 1- we should add voices , 2- we should adjuct the code for the cycles 